In [ ]:
import os, sys, math, gc
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import seaborn as sns
import lightgbm as lgb
import pickle as pkl
from utils.utils import merge_eval_sold_on_df, sort_df_on_d, WRMSSE, RMSSE, _down_cast, data_preprocessing, diff_lists, log_status
from utils.utils import customIter, cross_validation_on_validation_set, ensemble_submissions

from utils.configure_logger import configure_logger
configure_logger()
from logging import getLogger
logger = getLogger(__name__)

import warnings
warnings.simplefilter("ignore")

In [ ]:
DATA_BASE_PATH = '../data/m5-forecasting-accuracy/'
DATA_BASE_PATH_UNCERTAINTY = '../data/m5-forecasting-uncertainty/'
SALES_EVALUATION = 'sales_train_evaluation.csv'
SALES_VALIDATION = 'sales_train_validation.csv'
CALENDAR = 'calendar.csv'
SAMPLE_SUBMISSION = 'sample_submission.csv'
SELL_PRICES = 'sell_prices.csv'

PRECOMPUTED_BASE_PATH = '../data/uncertainty/features/'

DAYS: int = 28
QUANTILES: int = [0.005, 0.025, 0.165, 0.25, 0.50, 0.75, 0.835, 0.975, 0.995]

AGG_LEVEL_COLUMNS = {
    "Level1": [], # no grouping, sum of all
    "Level2": ['state_id'],
    "Level3": ['store_id'],
    "Level4": ['cat_id'],
    "Level5": ['dept_id'],
    "Level6": ['state_id', 'cat_id'],
    "Level7": ['state_id', 'dept_id'],
    "Level8": ['store_id', 'cat_id'],
    "Level9": ['store_id', 'dept_id'],
    "Level10": ['item_id'],
    "Level11": ['state_id', 'item_id'],
    "Level12": ['item_id','store_id'],
}